In [ ]:
# 公用模块

import math
import numpy as np
from typing import *

from build123d import *

In [ ]:
# 脚本参数

VISUALIZE = True
ASSEMBLE = False
EXPORT = True

if VISUALIZE:
    from ocp_vscode import *

In [ ]:
# 全局设计参数

UNIT = 19.

## 对PCB进行逆向

In [ ]:
__PCB_ORIGIN = np.array([10.6075, 19.1550])  # 本代码中使用kle中的原点为各个图形的原点。该值为该原点在pcb中的坐标
__SWITCHES_ROW1_LOCS_RAW = [  # x, y
    (20.1075, 35.7800), (39.1075, 35.7800), (58.1075, 31.0300), (77.1075, 28.6550), (96.1075, 31.0300), (115.1075, 33.4050),
]
__SWITCHES_THUMBS_LOCS_RAW = [  # x, y, r, w, h
    (86.6075, 88.6550, 0, 1, 1), (107.6075, 91.4050, -15, 1, 1), (129.8575, 95.1550, 60, 1.5, 1)
]

SWITCHES_SIZES = [
    *[(UNIT, UNIT) for _ in range(len(__SWITCHES_ROW1_LOCS_RAW) * 3)],
    *[(w * UNIT, h * UNIT) for _, _, _, w, h in __SWITCHES_THUMBS_LOCS_RAW]
]

SWITCHES_LOCS: List[Location] = []
for row in range(3):
    for x, y in __SWITCHES_ROW1_LOCS_RAW:
        SWITCHES_LOCS.append(Pos(x - __PCB_ORIGIN[0], -(y - __PCB_ORIGIN[1]) - row * UNIT))
for x, y, r, _, _ in __SWITCHES_THUMBS_LOCS_RAW:
    SWITCHES_LOCS.append(Pos(x - __PCB_ORIGIN[0], -(y - __PCB_ORIGIN[1])) * Rot(0, 0, r))

PLATE_HOLES_LOCS = [Pos(x, -y, 0) for x, y in [
    np.array([29.6075, 45.28]) - __PCB_ORIGIN,
    np.array([29.6075, 64.28]) - __PCB_ORIGIN,
    np.array([105.5755, 41.7655]) - __PCB_ORIGIN,
    np.array([119.3305, 89.1350]) - __PCB_ORIGIN
]]

PCB_BACKLEDS_LOCS = [Pos(x, -y, 0) for x, y in [
    np.array([39.1075, 45.28]) - __PCB_ORIGIN,
    np.array([39.1075, 64.28]) - __PCB_ORIGIN,
    np.array([77.1075, 38.155]) - __PCB_ORIGIN,
    np.array([77.1075, 76.155]) - __PCB_ORIGIN,
    np.array([115.1075, 42.905]) - __PCB_ORIGIN,
    np.array([115.1075, 80.905]) - __PCB_ORIGIN
]]

In [ ]:
# 如有需要，导出按键外边框以协助后续外形设计

# SWITCHES_BOUNDARY = Sketch() + [loc * Rectangle(*size) for loc, size in zip(SWITCHES_LOCS, SWITCHES_SIZES)]
# exporter = ExportSVG()
# exporter.add_layer('switches').add_shape(SWITCHES_BOUNDARY)
# exporter.write('switches.svg')

## 外形设计

In [ ]:
def import_svg(file_path: str):
    code, val = import_svg_as_buildline_code(file_path)
    exec(code)
    return locals()[val].line

In [ ]:
# 轴体参数

# 轴在定位板顶面之下的部分。用于在定位板上挖孔
SWITCH_BELOW_WIDTH, SWITCH_BELOW_DEPTH = (14.1, 5.)
SWITCH_CLICK_SPACE, SWITCH_CLICK_DIAMETER = (1.41, 15.)
SWITCH_CLICK_WIDTH, SWITCH_CLICK_WIDTH, SWITCH_CLICK_HEIGHT = (4., 3., SWITCH_BELOW_DEPTH - SWITCH_CLICK_SPACE)

# 轴在定位板顶面之上的部分。用于在外壳上为键帽留出空间
SWITCH_ABOVE_WIDTH, SWITCH_ABOVE_HEIGHT = (UNIT, 20.)

In [ ]:
# 定位板参数

PLATE_OFFSET = 10
PLATE_TILTING = 12.5

PLATE_THICKNESS = 5

PLATE_PLANE = Plane(
    Pos(0, 0, PLATE_OFFSET) * Rot(0, -PLATE_TILTING, 0) * Rectangle(1, 1).face()
)

In [ ]:
# 整体外形参数

SHELL_THICKNESS = 4
TOP_OFFSET = 8
TOP_CHAMFER = (2., 2.)

CASE_TOP_PLANE = Plane(PLATE_PLANE * Rectangle(1, 1).face().offset(TOP_OFFSET))

TOP = make_face(import_svg('design/top.svg'))
BOTTOM = make_face(import_svg('design/bottom.svg'))
POCKET = make_face(import_svg('design/pocket.svg'))

In [ ]:
# PCB和接口参数

PCB_THICKNESS = 1.6
PROMICRO_OFFSET = 4

USB_WIDTH = 13
USB_HEIGHT = 7
USB_TAPER = 5

TRRS_DIAMETER = 8
TRRS_TAPER = 45

PCB_PLANE = Plane(PLATE_PLANE * Rectangle(1, 1).face().offset(-5))

USB_LOC = PCB_PLANE * Pos(124.3, -8, 2.5 - 3.16 / 2) * Rot(-90, 0, 0)
TRRS_LOC = PCB_PLANE * Pos(133.552, -55.162, 5 / 2) * Rot(0, 90, 0)

PCB = make_face(import_svg('design/pcb.svg'))
OLED = make_face(import_svg('design/oled.svg'))

In [ ]:
# PCB和定位板的固定相关参数

PLATE_HOLE_DIAMETER = 2
PLATE_SPACER_DIAMETER = 4
PLATE_SPACER_DEPTH = 3

__BACK_PLATE_ANCHORS = [
    (loc.position.X, loc.position.Y)
    for loc in [*PLATE_HOLES_LOCS, *PCB_BACKLEDS_LOCS]
]
from scipy.spatial import ConvexHull
BACK_PLATE = offset(make_face(Polyline(*[
    __BACK_PLATE_ANCHORS[idx]
    for idx in ConvexHull(__BACK_PLATE_ANCHORS).vertices
], close=True)), 10)

In [ ]:
# 底座参数

FEET_OUTLINE_DIAMETER = 8
FEET_ENCHASE_HOLE_DIAMETER = 4
FEET_ENCHASE_HOLE_DEPTH = 3

__BOTTOM_INLINE = offset(BOTTOM, -max(SHELL_THICKNESS, FEET_OUTLINE_DIAMETER / 2))
FEET_LOCS = [
    Pos(vertex.X, vertex.Y, vertex.Z)
    for vertex in (__BOTTOM_INLINE.edges().sort_by(Axis.X)[0].vertices()
                   + __BOTTOM_INLINE.edges().sort_by(Axis.X)[-1].vertices())
]

In [ ]:
# 腕托参数

WRIST_TOP_PLANE = Plane(
    CASE_TOP_PLANE * Pos(0, 0, -TOP_CHAMFER[1])
    * Pos(48.1683, -123.4488, 0) * Rot(0, 0, -13.5578)
    * Rectangle(1, 1).face()
)

WRIST_BOTTOM = make_face(import_svg('design/wrist_bottom.svg'))
WRIST_TOP = make_face(import_svg('design/wrist_top.svg'))
WRIST_TOP_CURVE = Bezier((60, 0), (0, 0), (-30, 0), (-60, -12))
WRIST_THICKNESS = 3.

WRIST_REST = make_face(fillet(import_svg('design/wrist_rest.svg').vertices(), 5))
WRIST_REST_WIDTH1, WRIST_REST_WIDTH2 = 120., 74.
WRIST_REST_HEIGHT = 80.
WRIST_REST_THICKNESS1, WRIST_REST_THICKNESS2 = 12., 6.
WRIST_REST_TILTING = math.atan2(WRIST_REST_THICKNESS1 - WRIST_REST_THICKNESS2, WRIST_REST_HEIGHT / 2) * 180. / math.pi

## 生成外壳

In [ ]:
# 分上下两部分分别生成外壳实体

solid = loft([
    BOTTOM,
    CASE_TOP_PLANE * Pos(0, 0, -TOP_CHAMFER[1]) * TOP
], ruled=True)

shell = offset(solid, -SHELL_THICKNESS, openings=solid.faces().sort_by(Axis.Z)[0])

solid += loft([
    CASE_TOP_PLANE * Pos(0, 0, -TOP_CHAMFER[1]) * TOP,
    CASE_TOP_PLANE * offset(TOP, -TOP_CHAMFER[0], kind=Kind.INTERSECTION)
])

case_: Part = split(solid, PCB_PLANE, keep=Keep.TOP) + split(shell, PCB_PLANE, keep=Keep.BOTTOM)

# 减去开槽，形成按键围栏
case_ -= extrude(PLATE_PLANE * POCKET, TOP_OFFSET * 2)

# 减去PCB区域，保证PCB能正常安装
case_ -= extrude(PCB_PLANE * PCB, -20)

# 减去定位板按键孔位
__KAILH = Part() + (
    Plane.YX * Box(SWITCH_BELOW_WIDTH, SWITCH_BELOW_WIDTH, SWITCH_BELOW_DEPTH, align=(Align.CENTER, Align.CENTER, Align.MIN))
    + Plane.YX * Pos(0, 0, SWITCH_CLICK_SPACE) * Box(SWITCH_CLICK_DIAMETER, SWITCH_CLICK_WIDTH, SWITCH_CLICK_HEIGHT, align=(Align.CENTER, Align.CENTER, Align.MIN))
    + Plane.XY * Box(SWITCH_ABOVE_WIDTH, SWITCH_ABOVE_WIDTH, SWITCH_ABOVE_HEIGHT, align=(Align.CENTER, Align.CENTER, Align.MIN))
)
case_ -= [
    PLATE_PLANE * loc * __KAILH
    for loc in SWITCHES_LOCS
]

# 生成PCB定位孔
case_ -= [
    PCB_PLANE * loc * extrude(Circle(PLATE_SPACER_DIAMETER / 2), amount=PLATE_SPACER_DEPTH)
    for loc in PLATE_HOLES_LOCS
]  # M2铜柱
case_ -= [
    PLATE_PLANE * loc * (extrude(Circle(PLATE_HOLE_DIAMETER / 2), amount=-PLATE_THICKNESS)
                         + extrude(Circle(3.8 / 2), amount=-1.3, taper=45))
    for loc in PLATE_HOLES_LOCS
]  # M2沉头螺丝

# 减去主控和OLED
case_ -= extrude(PLATE_PLANE * OLED, -PLATE_THICKNESS * 2)

# 减去接口
case_ -= USB_LOC * extrude(Rectangle(USB_WIDTH, USB_HEIGHT), amount=30, taper=-USB_TAPER)
case_ -= TRRS_LOC * (
    Cylinder(TRRS_DIAMETER / 2, 30, align=[Align.CENTER, Align.CENTER, Align.MIN])
    + Pos(0, 0, 4) * extrude(Circle(TRRS_DIAMETER / 2) + Pos(0, 10, 0) * Rectangle(TRRS_DIAMETER, 20), amount=30, taper=-TRRS_TAPER)
)

# 生成底板连接孔
case_ += [
    extrude(loc * Circle(FEET_OUTLINE_DIAMETER / 2), until=Until.NEXT, target=case_)
    for loc in FEET_LOCS
]  # 生成孔位所在圆台
case_ -= [
    loc * Cylinder(FEET_ENCHASE_HOLE_DIAMETER / 2, FEET_ENCHASE_HOLE_DEPTH * 2)
    for loc in FEET_LOCS
]  # 挖孔

# 生成腕托磁铁槽
wrist_rest_mag_faces = (
    split(
        loft([
            BOTTOM,
            CASE_TOP_PLANE * Pos(0, 0, -TOP_CHAMFER[1]) * TOP
        ], ruled=True),
        PCB_PLANE, keep=Keep.BOTTOM
    )
    .faces().filter_by_position(Axis.Y, -math.inf, -57).filter_by_position(Axis.X, 19, 95))
wrist_rest_mag_planes = [
    Plane(face.center(), z_dir=face.normal_at())
    for face in wrist_rest_mag_faces
]
mag_pockets = [extrude(plane.offset(-0.5) * Circle(4), amount=-SHELL_THICKNESS) for plane in wrist_rest_mag_planes]
case_ = case_ - mag_pockets

# 完成建模
case_.label = 'case'

In [ ]:
if VISUALIZE:
    show_object(case_)

In [ ]:
if EXPORT:
    case_.export_step('build/case.step')

## 生成PCB背板

In [ ]:
backplate: Part = extrude(BACK_PLATE, -2) - [loc * Cylinder(PLATE_HOLE_DIAMETER / 2, 6) for loc in PLATE_HOLES_LOCS]
backplate.label = 'pcb_back'

In [ ]:
if EXPORT:
    backplate.export_step('build/back.step')

## 生成底板

In [ ]:
bottom: Part = extrude(BOTTOM, -2) - [loc * Cylinder(PLATE_HOLE_DIAMETER/2, 6) for loc in FEET_LOCS]
bottom.label = 'bottom'

In [ ]:
if EXPORT:
    bottom.export_step('build/bottom.step')

## 生成腕托

In [ ]:
# 腕托托架

wrist_rest_plate = loft([
    WRIST_BOTTOM,
    CASE_TOP_PLANE * Pos(0, 0, -TOP_CHAMFER[1]) * WRIST_TOP
], ruled=True)

__above = split(wrist_rest_plate, WRIST_TOP_PLANE.offset(-(10 + WRIST_THICKNESS)), keep=Keep.TOP)
__shell = offset(
    wrist_rest_plate, amount=-WRIST_THICKNESS,
    openings=wrist_rest_plate.faces().sort_by(Axis(WRIST_TOP_PLANE.origin, WRIST_TOP_PLANE.z_dir))[0]
)
__below = split(__shell, WRIST_TOP_PLANE.offset(-(10 + WRIST_THICKNESS)), keep=Keep.BOTTOM)

wrist_rest_plate = __above + __below

wrist_rest_plate = split(
    wrist_rest_plate,
    Plane(WRIST_TOP_PLANE * Rot(WRIST_REST_TILTING, 0, 0) * Rectangle(1, 1).face()),
    keep=Keep.BOTTOM
)

wrist_rest_plate -= WRIST_TOP_PLANE * extrude(WRIST_REST, amount=-10)

wrist_rest_plate.label = 'wrist rest plate'

In [ ]:
# 腕托本体

wrist_rest: Part = split(
    extrude(WRIST_REST, amount=12),
    Plane(Pos(0, 0, 12) * Rot(WRIST_REST_TILTING, 0, 0) * Rectangle(1, 1).face()),
    keep=Keep.BOTTOM
)

wrist_rest = fillet(
    wrist_rest.edges().filter_by(Axis.Z, reverse=True).filter_by_position(Axis.Z, 2, 20)
    - wrist_rest.edges().filter_by_position(Axis.Y, -10, 20),
    radius=1
)

wrist_rest.label = 'wrist rest'

In [ ]:
if VISUALIZE:
    show(wrist_rest_plate, WRIST_TOP_PLANE * Pos(0, 0, -10) * wrist_rest, case_)

In [ ]:
if EXPORT:
    wrist_rest_plate.export_step('build/wrist_rest.step')